In [1]:
# IPYTHON_OPTS="notebook spark.ipynb inline" ~/spark-1.6.0/bin/pyspark

gdata = {'a':{'f': 14}, 'a': {'c': 9}, 'a': {'b':7}, 'c' :{'d':  11}, 'c': {'f': 2}, 'c': {'a':9}, 'c': {'b': 10}, 'b': {'d': 15},'b': {'a': 7}, 'b': {'c': 10}, 'e': {'d': 6}, 'e': {'f': 9}, 'd': {'c': 11}, 'd': {'e': 6}, 'd': {'b':15}, 'f': {'a':14}, 'f': {'c': 2}, 'f': {'e': 9}}

In [46]:
import numpy as np
data = sc.textFile('/Users/ksiegler/Documents/USFClasses/Distcomp/edgelist.csv', minPartitions=10)

# data = sc.textFile('edgelist.csv')


def parse(line):
    pieces = line.split(',')
    from_ = int(pieces[0])
    to_ = int(pieces[1])
    weight = float(pieces[2])
    return {'from': from_, 'to': to_, 'weight': weight}

parsed = data.map(parse)
print(parsed.take(10))

[{'to': 1492123354, 'from': 1492123648, 'weight': 17.2556233905}, {'to': 1492123645, 'from': 1492123648, 'weight': 23.4996768929}, {'to': 1492123646, 'from': 1492123648, 'weight': 13.9847237894}, {'to': 1492123642, 'from': 1492123649, 'weight': 5.67260765562}, {'to': 1492123644, 'from': 1492123649, 'weight': 6.16630137613}, {'to': 2867855366, 'from': 2867855362, 'weight': 6.86085984941}, {'to': 2867855399, 'from': 2867855362, 'weight': 6.55430800833}, {'to': 2867855393, 'from': 2867855363, 'weight': 3.49327463699}, {'to': 2867855307, 'from': 2867855363, 'weight': 2.88642569255}, {'to': 2867855361, 'from': 2867855364, 'weight': 10.6862626623}]


In [16]:
nodes = parsed.map(lambda x: x['from']).distinct()
nodes.take(10)

[1492123648,
 2867855360,
 2867855362,
 2867855364,
 2867855366,
 2867855368,
 2867855370,
 2867855372,
 2009595906,
 2867855374]

In [17]:
dist = nodes.map(lambda x: (x, np.inf))
dist.take(10)

[(1492123648, inf),
 (2867855360, inf),
 (2867855362, inf),
 (2867855364, inf),
 (2867855366, inf),
 (2867855368, inf),
 (2867855370, inf),
 (2867855372, inf),
 (2009595906, inf),
 (2867855374, inf)]

In [19]:
unvisited = dist.collectAsMap()

In [20]:
start_node = 1237277400
end_node = 178755464


In [21]:
distance = 0

In [22]:
visited = dict()

In [23]:
visited[start_node] = 0

In [24]:
current_node = start_node

In [25]:
get_neighbs = parsed.filter(lambda x: x['from'] == current_node)

In [26]:
non_vis = get_neighbs.filter(lambda x: x['to'] in unvisited.keys())

In [28]:
get_weights = non_vis.collect()

AttributeError: 'list' object has no attribute 'take'

In [23]:
for i in range(len(get_weights)):
    new_distance = get_weights[i]['weight'] + distance
    unvisited[get_weights[i]['to']] = new_distance

In [24]:
del unvisited[current_node]

In [25]:
candidates = [node for node in unvisited.items()]

In [26]:
candidates.sort(key=lambda tup: tup[1])

In [27]:
current_node, distance = candidates[0]

In [28]:
current_node, distance

(u'b', 7)

In [48]:
def parse(line):
    pieces = line.split(',')
    from_ = int(pieces[0])
    to_ = int(pieces[1])
    weight = float(pieces[2])
    return {'from': from_, 'to': to_, 'weight': weight}
# from pyspark import SparkConf, SparkContext

# conf = SparkConf().setMaster("local[*]")
# sc   = SparkContext(conf=conf)
parsed = data.map(parse)
parsed.cache()
# parsed = sc.broadcast(parsed)
nodes = parsed.map(lambda x: x['from']).distinct()
dist = nodes.map(lambda x: (x, np.inf))
unvisited = dist.collectAsMap()
start_node = 1237277400
end_node = 2404284708
distance = 0
visited = dict()
visited[start_node] = 0
current_node = start_node
path = []
path.append(start_node)
import time
time1 = time.time()
print time
print(current_node, distance)
while current_node != end_node:
    get_neighbs = parsed.filter(lambda x: x['from'] == current_node, numPartitions = 6)
    non_vis = get_neighbs.filter(lambda x: x['to'] in unvisited.keys(), numPartitions = 6)
    get_weights = non_vis.collect()
    for i in range(len(get_weights)):
        new_distance = get_weights[i]['weight'] + distance
        if unvisited[get_weights[i]['to']] == np.inf or unvisited[get_weights[i]['to']]>new_distance:
            unvisited[get_weights[i]['to']] = new_distance
    del unvisited[current_node]
    candidates = [node for node in unvisited.items() if node[1]]
    candidates.sort(key=lambda tup: tup[1])
    current_node, distance = candidates[0]
    print(current_node, distance)
    path.append(current_node)
newtime = time.time()
total = newtime - time1
print("Starting from {0} and going to {1}, the path found by Dijkstra's algorithm is").format(start_node, end_node)
print(path)
print('with a cost of {0}.').format(distance)
print('with a time of {0} seconds').format(total)

<module 'time' from '/Users/ksiegler/anaconda/lib/python2.7/lib-dynload/time.so'>
(1237277400, 0)


TypeError: filter() got an unexpected keyword argument 'numPartitions'

In [30]:
current_node, distance

(u'e', 20)

In [31]:
unvisited

{u'd': 20, u'e': 20}

In [40]:
print(parsed.getNumPartitions())

2
